<a href="https://colab.research.google.com/github/jmarcosbs/regiao_de_congestao/blob/main/regiao_de_congestao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **📊 Indicador de região de congestão de preços em ativos financeiros**


> Bem-vindo ao meu primeiro projeto em Python, este é um projeto autêntico e estou estudando para enriquecê-lo com novas funções e futuramente técnicas de *machine learning*. O objetivo é que ele seja migrado para as plataformas do Metatrader e TradingView.


> Este indicador busca no gráfico de preços a principal região de congestão ou "região de briga" que são aquelas onde há mais aberturas e fechamentos de candles.


> O algoritmo começa comparando duas metades do range de preços e selecionando aquela que tem maior pontuação, depois armazena o novo range de preços da região com maior pontuação e analisa as duas metades dessa nova região. Usando o `while` a seleção se repete até que não haja distância entre as regiões.


> Você pode selecionar o ativo que quiser a partir da biblioteca do Yahoo Finance, além do timeframe e o intervalo dos dados. Basta alterar os valores de `tickers`, `period` e `interval`.


```
data = yf.download(tickers='EURJPY=x', period='1d', interval='5m')
```

> Será um prazer receber sua visita no meu [GitHub](https://https://github.com/jmarcosbs)











In [24]:
import pandas as pd
import pandas_datareader.data as pdr
import yfinance as yf
import plotly.graph_objects as go

In [66]:
#Definindo variáveis

data = yf.download(tickers='EURJPY=x', period='1d', interval='5m')

maxima = max(data['High'])

minima = min(data['Low'])

meio_regiao = (maxima + minima) / 2

aberturas = data['Open']

fechamentos = data['Close']

#--------------------------------------------------

#Definindo funções para calular pontuações e armazenar dados

#Função calcular pontuação região superior

def funcao_superior(aberturas, fechamentos, meio):

  qtd_aberturas = []
  qtd_fechamentos = []

  for preco in aberturas:
    if preco >= meio:
      qtd_aberturas.append(preco)

  for preco in fechamentos:
    if preco >= meio:
      qtd_fechamentos.append(preco)

  pts_superior = len(qtd_aberturas) + len(qtd_fechamentos)

  return {'pontuacao': pts_superior, 'qtd_aberturas': qtd_aberturas, 'qtd_fechamentos': qtd_fechamentos}

#--------------------------------------

#Função calcular pontuação da região inferior

def funcao_inferior(aberturas, fechamentos, meio):

  qtd_aberturas = []
  qtd_fechamentos = []

  for preco in aberturas:
    if preco <= meio:
      qtd_aberturas.append(preco)

  for preco in fechamentos:
    if preco <= meio:
      qtd_fechamentos.append(preco)

  pts_inferior = len(qtd_aberturas) + len(qtd_fechamentos)

  return {'pontuacao': pts_inferior, 'qtd_aberturas': qtd_aberturas, 'qtd_fechamentos': qtd_fechamentos}

#--------------------------------------------------

#--------------------------------------------------

#Criando o loop do robô para selecionar melhor região

numero_regiao = 0

while maxima - minima > 0:

  pontuacao_superior = funcao_superior(aberturas, fechamentos, meio_regiao)
  pontuacao_inferior = funcao_inferior(aberturas, fechamentos, meio_regiao)

  #Define melhores regiões e armazena novos dados

  if pontuacao_superior['pontuacao'] >= pontuacao_inferior['pontuacao']:

    numero_regiao += 1
    aberturas = pontuacao_superior['qtd_aberturas']
    fechamentos = pontuacao_superior['qtd_fechamentos']
    maxima = max(aberturas and fechamentos)
    minima = min(aberturas and fechamentos)
    meio_regiao = (maxima + minima) / 2

    display(f"#{numero_regiao} Região superior escolhida {pontuacao_superior['pontuacao']}")

  elif pontuacao_superior['pontuacao'] < pontuacao_inferior['pontuacao']:

    numero_regiao += 1
    aberturas = pontuacao_inferior['qtd_aberturas']
    fechamentos = pontuacao_inferior['qtd_fechamentos']
    maxima = max(aberturas and fechamentos)
    minima = min(aberturas and fechamentos)
    meio_regiao = (maxima + minima) / 2

    display(f"#{numero_regiao} Região inferior escolhida {pontuacao_inferior['pontuacao']}")

#--------------------------------------------------

display(f"Região de congestão: {meio_regiao:.2f}")


# Plotagem do gráfico

fig = go.Figure(data=[go.Candlestick(x=data.index, open=data['Open'], high=data['High'], low=data['Low'], close=data['Close'])])
fig.add_hline(y=maxima)
fig.add_hline(y=minima)
fig.add_hline(y=meio_regiao)
fig.show()

#--------------------------------------------------



[*********************100%%**********************]  1 of 1 completed


'#1 Região superior escolhida 343'

'#2 Região inferior escolhida 238'

'#3 Região superior escolhida 138'

'#4 Região inferior escolhida 78'

...

'#6 Região inferior escolhida 30'

'#7 Região superior escolhida 19'

'#8 Região superior escolhida 11'

'#9 Região superior escolhida 6'

'Região de congestão: 157.90'